In [46]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key=os.getenv("GROQ")

from langchain_groq import ChatGroq

In [47]:
model = ChatGroq(model="llama-3.1-8b-instant",groq_api_key=groq_api_key)


In [48]:
from langchain_core.messages import HumanMessage

response = model.invoke([
    HumanMessage(content="Hi, my name is Prudhvi. I'm a software developer learning GenAI.")
])

print(response.content)


Nice to meet you, Prudhvi. I'm happy to help you learn and explore the world of GenAI (Generative Artificial Intelligence). It's a rapidly evolving field with many exciting applications in software development, natural language processing, and more.

What specific aspects of GenAI would you like to learn about or discuss? Are you interested in:

1. Text-to-text or text-to-image models?
2. Model architecture (e.g., transformers, recurrent neural networks)?
3. Data preprocessing and generation techniques?
4. Applications of GenAI in software development (e.g., code generation, chatbots)?
5. Something else?

Let me know, and I'll do my best to assist you in your learning journey, Prudhvi.


#### Message History


We can use a Message History class to wrap our model and make it statefull.This will keep track of inputs and outputs of the model and store them in some datastore. Fututre intecrations will then load those messages and pss them into the chain as part of the input.

In [49]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:id) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [50]:
config={"configurable":{"session_id":"chat1"}}


response=with_message_history.invoke(
    [HumanMessage(content="Hi, my name is Prudhvi. I'm a software developer learning GenAI.")],
    config=config
)

In [51]:
response.content

'Hello Prudhvi, nice to meet you. Learning GenAI (Generative AI) can be a fascinating and rewarding field. GenAI involves the development of algorithms and models that can generate new, original content, such as text, images, music, or even entire stories.\n\nWhat specific aspects of GenAI are you interested in learning? Are you looking at text generation, image generation, or perhaps something else?'

In [52]:
response2=with_message_history.invoke(
    [HumanMessage(content="What is my name and iam trying to learn")],
    config=config
)

response2.content

"Your name is Prudhvi, and you're trying to learn about GenAI (Generative AI)."

In [53]:
### what if we change the session id
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content


"I don't have any information about your name. I'm a conversational AI, and this is the beginning of our conversation. I don't store personal data, so I won't be able to recall your name if you don't tell me. If you'd like to share your name, I'm happy to chat with you!"

#### Prompt templates

promt template help to turn raw user inforamtion into a formate that the LLM can work with. in this case the raw user input is just a mesage which weare passiong to LLm let now make that a bit more complicated. 

First add in a sustem messsage with some cistom instructions we will add in more inputs besides just the messages.


In [54]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Amnswer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [55]:
chain.invoke({"messages":[HumanMessage(content="Hi my name prudhvi")]})

AIMessage(content="Hello Prudhvi, it's nice to meet you. I'm here to assist you with any questions or information you may need. What's on your mind today? Do you have a specific topic or issue you'd like to discuss?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 59, 'total_tokens': 109, 'completion_time': 0.059819112, 'completion_tokens_details': None, 'prompt_time': 0.003091247, 'prompt_tokens_details': None, 'queue_time': 0.005378352, 'total_time': 0.062910359}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_9ca2574dca', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b8875-6324-71e2-bf0a-c9fa93d75a43-0', usage_metadata={'input_tokens': 59, 'output_tokens': 50, 'total_tokens': 109})

In [56]:
with_message_history2=RunnableWithMessageHistory(chain,get_session_history)

In [57]:
config={"configurable":{"session_id":"chat3"}}
response=with_message_history2.invoke(
    [HumanMessage(content="Hi my name is prudhvi")],
    config=config
)

response.content

"Hello Prudhvi. It's nice to meet you. Is there something I can help you with today?"

In [58]:
response = with_message_history2.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Prudhvi.'

#### Managing the conversation History

One importatnt concept to understand when building chatbot is how to manage conversation history. if left unmanged the list of messages will grow unbounded and potentially overflow the context window of the LLM. Thereefore it is importatnt to add a step that limits the size of the messages you are passing in. trim_messages to reduce how many messages we are sending to the model. 